In [ ]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import LabelEncoder,OneHotEncoder

In [ ]:
col_names = ["duration","protocol_type","service","flag","src_bytes",
    "dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
    "logged_in","num_compromised","root_shell","su_attempted","num_root",
    "num_file_creations","num_shells","num_access_files","num_outbound_cmds",
    "is_host_login","is_guest_login","count","srv_count","serror_rate",
    "srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
    "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
    "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
    "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
    "dst_host_rerror_rate","dst_host_srv_rerror_rate","label"]
train_df = pd.read_csv("KDDTrain+_2.csv", header=None, names = col_names)
test_df = pd.read_csv("KDDTest+_2.csv", header=None, names = col_names)

In [ ]:
train_df.head()

In [ ]:
train_df.shape

In [ ]:
train_df.describe()

In [ ]:
for col in train_df.columns:
    if train_df[col].dtypes == 'object' :
        category = len(train_df[col].unique())
        print("'{col}' : {category}".format(col=col, category=category))

In [ ]:
columns_cat=['protocol_type', 'service', 'flag'] 
train_df_categorical_values = train_df[columns_cat]
test_df_categorical_values = test_df[columns_cat]
train_df_categorical_values.head()

In [ ]:
unique_protocol=sorted(train_df.protocol_type.unique())
string1 = 'Protocol_type_'
unique_protocol2=[string1 + x for x in unique_protocol]
unique_service=sorted(train_df.service.unique())
string2 = 'service_'
unique_service2=[string2 + x for x in unique_service]
unique_flag=sorted(train_df.flag.unique())
string3 = 'flag_'
unique_flag2=[string3 + x for x in unique_flag]
dumcols=unique_protocol2 + unique_service2 + unique_flag2

In [ ]:
unique_service_test=sorted(test_df.service.unique())
unique_service2_test=[string2 + x for x in unique_service_test]
testdumcols=unique_protocol2 + unique_service2_test + unique_flag2

In [ ]:
train_df_categorical_values_lblenc=train_df_categorical_values.apply(LabelEncoder().fit_transform)
train_df_categorical_values_lblenc.head()

In [ ]:
test_df_categorical_values_lblenc=test_df_categorical_values.apply(LabelEncoder().fit_transform)
test_df_categorical_values_lblenc.head()

In [ ]:
oneHotenc = OneHotEncoder()
train_df_categorical_values_OHenc = oneHotenc.fit_transform(train_df_categorical_values_lblenc)
train_df_cat = pd.DataFrame(train_df_categorical_values_OHenc.toarray(),columns=dumcols)
# test set
test_df_categorical_values_OHenc = oneHotenc.fit_transform(test_df_categorical_values_lblenc)
test_df_cat = pd.DataFrame(test_df_categorical_values_OHenc.toarray(),columns=testdumcols)

train_df_cat.head()

In [ ]:
trainservice=train_df['service'].tolist()
testservice= test_df['service'].tolist()
difference=list(set(trainservice) - set(testservice))
string = 'service_'
difference=[string + x for x in difference]
difference

In [ ]:
for col in difference:
    test_df_cat[col] = 0

In [ ]:
newdf_train=train_df.join(train_df_cat)
newdf_train.drop('flag', axis=1, inplace=True)
newdf_train.drop('protocol_type', axis=1, inplace=True)
newdf_train.drop('service', axis=1, inplace=True)


In [ ]:
newdf_test=test_df.join(test_df_cat)
newdf_test.drop('flag', axis=1, inplace=True)
newdf_test.drop('protocol_type', axis=1, inplace=True)
newdf_test.drop('service', axis=1, inplace=True)

In [ ]:
newdf_train.shape

In [ ]:
newdf_test.shape

In [ ]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.callbacks import EarlyStopping

In [ ]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
newdf_train['label'] = le.fit_transform(newdf_train['label'])

In [ ]:
y_train_df = newdf_train['label']

In [ ]:
newdf_test['label'] = le.fit_transform(newdf_test['label'])

In [ ]:
y_test_df = newdf_test['label']

In [ ]:
x_test_df = newdf_test.drop('label',axis=1)

In [ ]:
x_train_df = newdf_train.drop('label',axis=1)

In [ ]:
x_test_df.head()

In [ ]:
x_train =x_train_df.as_matrix(x_train_df.columns).astype(np.float32)

In [ ]:
#y_train_df.head()
y_train = y_train_df.as_matrix().astype(np.float32)

In [ ]:
x_test = x_test_df.as_matrix(x_test_df.columns).astype(np.float32)
y_test = y_test_df.as_matrix().astype(np.float32)

In [ ]:
model = Sequential()
model.add(Dense(10, input_dim=x_train_df.shape[1], kernel_initializer='normal', activation='relu'))
model.add(Dense(50, input_dim=x_train_df.shape[1], kernel_initializer='normal', activation='relu'))
model.add(Dense(10, input_dim=x_train_df.shape[1], kernel_initializer='normal', activation='relu'))
model.add(Dense(1, kernel_initializer='normal'))
model.add(Dense(y_train_df.shape[1],activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')
model.fit(x_train,y_train,validation_data=(x_test,y_test),callbacks=[monitor],verbose=2,epochs=1000)

In [ ]:
pred = model.predict(x_test)
pred = np.argmax(pred,axis=1)
eval = np.argmax(y_test,axis=1)
score = metrics.accuracy_score(eval, pred)